In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.losses import MSE
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model
import tensorflow_hub as hub

import get_dataset as kws_data
import kws_util
import argparse
from tqdm import tqdm

## Import Dataset

In [ ]:
Flags, unparsed = kws_util.parse_command()
Flags.window_size_ms=40.0
Flags.window_stride_ms=20.0
Flags.batch_size = 1
Flags.data_dir = '/home/nesl/209as_sec/audio_ks/data'
print('We will download data to {:}'.format(Flags.data_dir))
ds_train, ds_test, ds_val = kws_data.get_training_data(Flags)
print("Done getting data")
train_shuffle_buffer_size = 85511
val_shuffle_buffer_size = 10102
test_shuffle_buffer_size = 4890

ds_train = ds_train.shuffle(train_shuffle_buffer_size)
ds_val = ds_val.shuffle(val_shuffle_buffer_size)
ds_test = ds_test.shuffle(test_shuffle_buffer_size)

ds_train = ds_train.map(lambda x, y: (x, tf.one_hot(y, depth=12)))
ds_val = ds_val.map(lambda x, y: (x, tf.one_hot(y, depth=12)))
ds_test = ds_test.map(lambda x, y: (x, tf.one_hot(y, depth=12)))

## Load Model

In [ ]:
model = load_model('/home/nesl/209as_sec/audio_ks/Big Models/CNN/cnn_large.h5')
model.summary()

## Attack

In [ ]:
def pgd_attack(model,iterations, image, label, alpha, eps):
    gen_img = tf.identity(image)
    gen_img = gen_img + tf.random.uniform(gen_img.get_shape().as_list(), minval=-eps, 
                                          maxval=eps, dtype=tf.dtypes.float32)
    x_temp = image
    for iter in range(iterations):
        imgv = tf.Variable(gen_img)
        with tf.GradientTape() as tape:
            tape.watch(imgv)
            predictions = model(imgv)
            loss = tf.keras.losses.CategoricalCrossentropy()(label, predictions)
            grads = tape.gradient(loss,imgv)
        signed_grads = tf.sign(grads)
        gen_img = gen_img + (alpha*signed_grads)
        gen_img = tf.clip_by_value(gen_img, image-eps, image+eps)
    return gen_img

In [ ]:
eps = [0.1,0.3,0.5,0.7,0.9,1.0,2.0]
iterations = 5
alpha = [0.1,0.3,0.5,0.7,0.9,1.0]
take_size = 4890
accu_num = []
eps_list = []
alpha_list = []

for al in alpha:
    for item in eps:
        countadv = 0
        for image, label in tqdm(ds_test.take(take_size)):
            imageLabel = np.array(label).argmax()
            imagePred = model.predict(image)
            imagePred = imagePred.argmax()
            adversary = pgd_attack(model,iterations,image, label, alpha=al, eps=item)
            pred = model.predict(adversary)
            adversaryPred = pred[0].argmax()
            if imagePred == adversaryPred:
                countadv += 1
            
        print("Adversarial accuracy : ", countadv / take_size)
        accu_num.append(countadv / take_size)
        eps_list.append(item)
        alpha_list.append(al)

In [ ]:
accu_num

In [ ]:
eps_list

In [ ]:
alpha_list